In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#Load the ANN trained model, Pickle files: scalar,onehot, label

In [3]:
model=load_model("model.h5")

In [4]:
with open("label_encoder_gender.pkl","rb") as file:
    label_encoder_gender=pickle.load(file)

with open("one_hot_encoder_geo.pkl","rb") as file:
    one_hot_encoder_geo=pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler=pickle.load(file)

In [5]:
#Example input data
input_data={
    "CreditScore":600,
    "Geography": "France",
    "Gender":"Male",
    "Age": 40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [19]:
geo_encoded=one_hot_encoder_geo.transform([[input_data["Geography"]]])
geo_encoded_df=pd.DataFrame(geo_encoded, columns= one_hot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

C:\Users\Vikrant Vivek Deo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
input_df=pd.DataFrame([input_data])

In [28]:
#Encode Categorical Variables:
input_df["Gender"]=label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [30]:
input_df["Gender"].unique()

array([1])

In [33]:
#Concatenation with one_hot_encoded data and label_encoded geography:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [34]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
#Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [37]:
#Predict Churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


array([[0.10277669]], dtype=float32)

In [38]:
prediction_probability=prediction[0][0]

In [39]:
prediction_probability

0.10277669

In [41]:
if prediction_probability>0.5:
    print("The customer is going to churn.")
else:
    print("The customer is not going to churn.")

The customer is not going to churn.


In [46]:
one_hot_encoder_geo.categories_[::]

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [47]:
one_hot_encoder_geo.categories_[0]
#We used this in our streamlit app selectbox to select the geography

array(['France', 'Germany', 'Spain'], dtype=object)

In [48]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)